In [3]:
import collections
import os
from os.path import join
import sys
import random
import torch
import numpy as np
import pandas as pd
from torch.utils import data
from torch.utils.data import Dataset, DataLoader
from scipy.sparse import csr_matrix
import scipy.sparse as sp
from time import time
from random import sample

DATA_PATH = 'data/SL/'
KG_FILE_NAME = 'raw/kg2id.txt'
SL_DATASET = 'raw/test_d.txt'
NEIGHBOR_NUM = 16

In [9]:
sl_df = pd.read_csv(SL_DATASET, sep='\t')
all_a = np.array(sl_df['gene_a'])
all_b = np.array(sl_df['gene_b'])
all_label=np.array(sl_df['label'])
shuffle_idx = np.random.permutation(np.arange(len(all_a)))
all_a = all_a[shuffle_idx]
all_b = all_b[shuffle_idx]
all_label=all_label[shuffle_idx]

In [10]:
for i in range(5):
    fold_n=i+1
    len_per_fold = len(all_a) // 5
    all_idx = [i for i in range(len(all_a))]
    val_test_idx = [
            i for i in range((fold_n - 1) * len_per_fold, fold_n *
                             len_per_fold)
        ]
    train_idx = list(set(all_idx) - set(val_test_idx))
    train_a = all_a[train_idx]
    train_b = all_b[train_idx]
    train_label=all_label[train_idx]
    val_a = all_a[val_test_idx[:len_per_fold // 2]]
    val_b = all_b[val_test_idx[:len_per_fold // 2]]
    val_label=all_label[val_test_idx[:len_per_fold // 2]]
    test_a = all_a[val_test_idx[len_per_fold // 2:]]
    test_b = all_b[val_test_idx[len_per_fold // 2:]]
    test_label=all_label[val_test_idx[len_per_fold // 2:]]
    train_df=pd.DataFrame()
    train_df['gene_a']=train_a
    train_df['gene_b']=train_b
    train_df['label']=train_label
    val_df=pd.DataFrame()
    val_df['gene_a']=val_a
    val_df['gene_b']=val_b
    val_df['label']=val_label
    test_df=pd.DataFrame()
    test_df['gene_a']=test_a
    test_df['gene_b']=test_b
    test_df['label']=test_label
    train_df.to_csv('fold_data/fold_'+str(i+1)+'/train.txt',sep='\t',header=True,index=False)
    val_df.to_csv('fold_data/fold_'+str(i+1)+'/val.txt',sep='\t',header=True,index=False)
    test_df.to_csv('fold_data/fold_'+str(i+1)+'/test.txt',sep='\t',header=True,index=False)

    
    